In [75]:
import json
import torch
import pickle
import transformers

import numpy as np
import pandas as pd

In [76]:
from pathlib import Path
from itertools import chain
from tqdm import tqdm

In [77]:
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import BertTokenizer
from transformers import BertModel

In [78]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

## Data

In [79]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len
        
        self.comment_text = dataframe['内容']    # X
        self.targets = self.data['标签']    # y(s)
        

    def __len__(self):
        return len(self.comment_text)
    

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split()) # split, 默认为所有的空字符，包括空格、换行(\n)、制表符(\t)等

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            # pad_to_max_length=True,  # @deprecated
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask'] # sentence 有效token位置掩码
        token_type_ids = inputs["token_type_ids"] # 多sentence(s)合并为一个sentence时，不同sentence的掩码


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            
            'targets': torch.tensor(self.targets[index], dtype=torch.float) # y(s) 没有特别处理
        }

In [80]:
def get_basic(p):
    with p.open() as f:
        d = json.load(f)
        d['标签'] = d['标签'].keys()
        return d

In [81]:
def get_comment(d, tag, comment):
    comment['标签'] = tag
    del comment['回复']
    comment.update(d)
    return comment

In [82]:
def get_comments(p):
    with p.open() as f:
        d = json.load(f)
        tag_comments = d['标签']
        del d['标签']
        return [get_comment(d, tag, comment) for tag, comments in tag_comments.items() for comment in comments]

### load

In [83]:
path_root = Path('./data')

In [84]:
path_data = Path('comment.taptap-20210203-1')

In [85]:
path = path_root/path_data

In [86]:
%time df_basic = pd.DataFrame([get_basic(p) for p in path.glob('*.json')])
print(df_basic.shape)
df_basic.head(1)

CPU times: user 14.8 s, sys: 4.57 s, total: 19.4 s
Wall time: 19.3 s
(150, 5)


,游戏名,游戏评分,游戏url,评论数量,标签
0,精灵契约,6.8,https://www.taptap.com/app/142111,870,"(过于氪金, 体验不错, 画面优良, 运营不足, 有趣好玩, 玩家互动多, 抽卡概率低, 厂..."


In [87]:
%time df_comments = pd.DataFrame(chain.from_iterable([get_comments(p) for p in tqdm(path.glob('*.json'))]))
print(df_comments.shape)
df_comments.head(1)

150it [00:15,  9.89it/s]


CPU times: user 13.8 s, sys: 3.74 s, total: 17.6 s
Wall time: 17.5 s
(488452, 14)


,用户名,评论时间,游戏评分,游戏时长,内容,手机型号,欢乐,点赞,点踩,回复量,标签,游戏名,游戏url,评论数量
0,Foo云少,2021-01-19 10:07:37,6.8,0,不好玩太氪金了我有段时间没有玩号在那个区都不知道了而且这个游戏刚刚出来的时候玩的10区的,华为畅享9 Plus,0,0,0,0,过于氪金,精灵契约,https://www.taptap.com/app/142111,870


In [88]:
df_comments_topic = df_comments[~df_comments.标签.isin(['好评', '中评', '差评', 'android', 'ios', 'web', '有游戏时长'])]
print(df_comments_topic.shape)
df_comments_topic.head(1)

(45450, 14)


,用户名,评论时间,游戏评分,游戏时长,内容,手机型号,欢乐,点赞,点踩,回复量,标签,游戏名,游戏url,评论数量
0,Foo云少,2021-01-19 10:07:37,6.8,0,不好玩太氪金了我有段时间没有玩号在那个区都不知道了而且这个游戏刚刚出来的时候玩的10区的,华为畅享9 Plus,0,0,0,0,过于氪金,精灵契约,https://www.taptap.com/app/142111,870


In [89]:
%time df_comments_topics = df_comments_topic[['内容', '标签']].groupby(['内容']).agg(list).reset_index()
print(df_comments_topics.shape)
df_comments_topics.head(3)

CPU times: user 992 ms, sys: 4 ms, total: 996 ms
Wall time: 992 ms
(30378, 2)


,内容,标签
0,\n\n\n玩过两个赛季的人，游戏消遣还不错，不过卡池锁卡劝退，一赛季初期获得刘备张飞，二赛...,[运营不足]
1,\n\n10:30开始进游戏就各种bug，各种系统维护。下午玩了不到一个小时，服务器又崩。无...,[运营不足]
2,"\n\n游戏模式还是挺有意思，可以自己铺路建设,把不同文明的帝国从几个人发展到上万人建成自己...","[体验不错, 有趣好玩]"


In [90]:
df_comments_topics['标签'].apply(len).value_counts()

1    20056
2     7069
3     2228
4      694
5      222
6       83
7       20
8        6
Name: 标签, dtype: int64

In [91]:
df_comments_topics[df_comments_topics['标签'].apply(len) > 1].head(3)

,内容,标签
2,"\n\n游戏模式还是挺有意思，可以自己铺路建设,把不同文明的帝国从几个人发展到上万人建成自己...","[体验不错, 有趣好玩]"
4,\n\n玩了差不多一个月了，这两天游戏里的汉家松鼠让我评论，那么我就来评一评。游戏立意不错，...,"[体验不错, 值得花钱, 厂商良心]"
5,\n\n这款游戏整体来说挺好的，剧情画风都挺不错，但是还是有一些问题。\n①难度设置的有些不...,"[剧情丰富, 画面优良]"


In [92]:
mlb = MultiLabelBinarizer()
# df_comments_topics.loc[:,'标签'] = mlb.fit_transform(df_comments_topics['标签']).tolist()
df_comments_topics['标签'] = mlb.fit_transform(df_comments_topics['标签']).tolist()
print(df_comments_topics.shape)
df_comments_topics.head(3)

(30378, 2)


,内容,标签
0,\n\n\n玩过两个赛季的人，游戏消遣还不错，不过卡池锁卡劝退，一赛季初期获得刘备张飞，二赛...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,\n\n10:30开始进游戏就各种bug，各种系统维护。下午玩了不到一个小时，服务器又崩。无...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"\n\n游戏模式还是挺有意思，可以自己铺路建设,把不同文明的帝国从几个人发展到上万人建成自己...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [93]:
df_comments_topics[df_comments_topics['标签'].apply(lambda x: 1 if int == type(x) else len(x)) > 1].head(3)

,内容,标签
0,\n\n\n玩过两个赛季的人，游戏消遣还不错，不过卡池锁卡劝退，一赛季初期获得刘备张飞，二赛...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,\n\n10:30开始进游戏就各种bug，各种系统维护。下午玩了不到一个小时，服务器又崩。无...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"\n\n游戏模式还是挺有意思，可以自己铺路建设,把不同文明的帝国从几个人发展到上万人建成自己...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [94]:
mlb.classes_

array(['IP还原差', 'UI体验好', 'UI体验差', '上手难度大', '优化相关', '体验不错', '体验较差', '值得花钱',
       '剧情丰富', '剧情单调', '厂商不给力', '厂商良心', '太肝了', '尊重原著', '平衡性好', '平衡性差',
       '广告太多', '广告影响小', '抄袭嫌疑', '护肝', '抽卡概率低', '抽卡概率高', '操作简单', '操作麻烦',
       '新手友好', '有创新', '有趣好玩', '玩家互动多', '玩家互动少', '玩法较差', '画面优良', '画面粗糙',
       '福利好', '福利差', '自由度低', '自由度高', '过于氪金', '运营不足', '运营给力', '配置要求低',
       '配置要求高', '音效很棒', '音效较差'], dtype=object)

In [95]:
with open('./model/mlb.pkl', 'wb') as f:
    pickle.dump(mlb, f)

### eda

#### content length

In [21]:
df_comments_length = df_comments_topics['内容'].apply(len)

In [22]:
df_comments_length.describe()

count    30378.000000
mean       183.025446
std        286.144866
min         21.000000
25%         44.000000
50%         87.000000
75%        203.000000
max      13714.000000
Name: 内容, dtype: float64

In [23]:
k = 1
mu_1std = df_comments_length.mean(axis=0) + k * df_comments_length.std(axis=0) 

print(df_comments_length.shape)
print(df_comments_length[df_comments_length < mu_1std].shape)
print(df_comments_length[df_comments_length < mu_1std].shape[0] / df_comments_length.shape[0])

(30378,)
(27670,)
0.9108565409177695


#### topic

In [24]:
tags = set(chain.from_iterable(df_basic.标签.apply(list).tolist()))
print(len(tags))
tags

50


{'IP还原差',
 'UI体验好',
 'UI体验差',
 'android',
 'ios',
 'web',
 '上手难度大',
 '中评',
 '优化相关',
 '体验不错',
 '体验较差',
 '值得花钱',
 '剧情丰富',
 '剧情单调',
 '厂商不给力',
 '厂商良心',
 '太肝了',
 '好评',
 '尊重原著',
 '差评',
 '平衡性好',
 '平衡性差',
 '广告太多',
 '广告影响小',
 '抄袭嫌疑',
 '护肝',
 '抽卡概率低',
 '抽卡概率高',
 '操作简单',
 '操作麻烦',
 '新手友好',
 '有创新',
 '有游戏时长',
 '有趣好玩',
 '玩家互动多',
 '玩家互动少',
 '玩法较差',
 '画面优良',
 '画面粗糙',
 '福利好',
 '福利差',
 '自由度低',
 '自由度高',
 '过于氪金',
 '运营不足',
 '运营给力',
 '配置要求低',
 '配置要求高',
 '音效很棒',
 '音效较差'}

In [25]:
df_comments.drop_duplicates('内容').shape

(182070, 14)

In [26]:
df_comments_topics.标签.apply(len).value_counts()

43    30378
Name: 标签, dtype: int64

#### +1/0/-1

In [27]:
df_comments_pnn = df_comments[df_comments.标签.isin(['好评', '中评', '差评'])]
print(df_comments_pnn.shape)
df_comments_pnn.head(1)

(195017, 14)


,用户名,评论时间,游戏评分,游戏时长,内容,手机型号,欢乐,点赞,点踩,回复量,标签,游戏名,游戏url,评论数量
1369,白鳞小蛇,2021-01-26 11:37:59,6.8,0,还不错，就是太依赖于抽卡，没什么英雄搭配\r\n\r\n这游戏凉了么？\r\n,,1,0,0,0,好评,精灵契约,https://www.taptap.com/app/142111,870


In [28]:
df_comments_pnn.drop_duplicates('内容').shape

(173610, 14)

In [29]:
df_comments_pnn.标签.value_counts(normalize=True)

好评    0.591400
差评    0.291764
中评    0.116836
Name: 标签, dtype: float64

### to dataset

In [30]:
int(mu_1std)

469

In [31]:
# MAX_LEN = 128
# MAX_LEN = int(mu_1std)
MAX_LEN = 512

TRAIN_BATCH_SIZE = 4 # 8 # 4
VALID_BATCH_SIZE = 4 # 8 # 4
EPOCHS = 20
LEARNING_RATE = 1e-05

In [32]:
new_df = df_comments_topics

In [33]:
# split train & test

train_size = 0.8

train_dataset = new_df.sample(frac=train_size, random_state=200)
test_dataset = new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (30378, 2)
TRAIN Dataset: (24302, 2)
TEST Dataset: (6076, 2)


In [ ]:
# model_name = 'bert-base-chinese'

# model_name = 'hfl/chinese-bert-wwm'
model_name = 'hfl/chinese-bert-wwm-ext'

# model_name = 'hfl/chinese-roberta-wwm-ext'
# model_name = 'hfl/chinese-roberta-wwm-ext-large'    # not working

# model_name = 'hfl/chinese-macbert-base'

# model_name = 'ckiplab/bert-base-chinese'

# model_name = 'ckiplab/albert-tiny-chinese'    # not working

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN) # diff MultiLabelDataset
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

## Model

In [ ]:
x = pipeline('a')
x('wgoewno nowgno')

In [72]:
class BERTClass(torch.nn.Module):
    
    
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained(model_name)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, len(mlb.classes_))
    
    
    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1.pooler_output)
        output = self.l3(output_2)
        return output

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def train(model, optimizer, training_loader, epoch, device):
    model.train()
    
    for _, data in enumerate(training_loader, 0): # enumerate(iterable, start=0)
        
        # _, by batch?
        
        # X
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        
        # y
        targets = data['targets'].to(device, dtype = torch.float)

        # train
        outputs = model(ids, mask, token_type_ids)

        # optimizer clean
        optimizer.zero_grad()
        
        # loss
        loss = loss_fn(outputs, targets)
        if 0 == _ % 5000:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        # TODO: why?
        # optimizer.zero_grad()
        
        # optimize
        loss.backward() # get new gradient, upon zero grad
        optimizer.step()

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
def validation(model, epoch):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]
    
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            
            # X
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            
            # y
            targets = data['targets'].to(device, dtype = torch.float)
            
            # pred
            outputs = model(ids, mask, token_type_ids)
            
            # TODO:
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            
    return fin_outputs, fin_targets

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [73]:
# model = BERTClass()

model_file = './model/chinese-bert-wwm-ext-4.bin'
model = torch.load(model_file)

In [ ]:
model.to(device);

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
%time for epoch in range(EPOCHS): train(model, optimizer, training_loader, epoch, device)

In [ ]:
for epoch in range(EPOCHS):
    
    outputs, targets = validation(model, epoch)
    outputs = np.array(outputs) >= 0.5
    
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    
    # TODO:
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    
    val_hamming_loss = metrics.hamming_loss(targets, outputs)
    val_hamming_score = hamming_score(np.array(targets), np.array(outputs))

    print(f"Hamming Score = {val_hamming_score}")
    print(f"Hamming Loss = {val_hamming_loss}")
    
    print(classification_report(targets, outputs, target_names=mlb.classes_))

In [ ]:
# Saving the files for inference

output_model_file = './model/chinese-bert-wwm-ext-5.bin'
# output_vocab_file = './model/vocab_chinese-bert-wwm-ext-1.bin'

torch.save(model, output_model_file)
# tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

## Ref

[1] https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

[2] https://huggingface.co/hfl/chinese-bert-wwm <br>
[3] https://huggingface.co/hfl/chinese-bert-wwm-ext <br>

[4] https://huggingface.co/hfl/chinese-roberta-wwm-ext <br>
[5] https://huggingface.co/hfl/chinese-roberta-wwm-ext-large <br>

[6] https://huggingface.co/hfl/chinese-macbert-base

[7] https://huggingface.co/ckiplab/bert-base-chinese <br>
[8] https://huggingface.co/ckiplab/albert-tiny-chinese